In [ ]:
import pickle
import numpy as np
import xarray
import rioxarray
with open('/Users/zachkeskinen/Documents/spicy-snow/data/main_test.pkl', 'rb') as f:
    ds = pickle.load(f)

In [ ]:
ds

In [ ]:
np.unique(ds.relative_orbit)

In [ ]:
ds = ds.set_xindex('relative_orbit')

In [ ]:
ds.sel(relative_orbit = 20)

In [ ]:
ds.where(ds.flight_dir == 'ascending', drop = True)